In [1]:
import tensorflow as tf
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2

# image resizing

In [64]:
search_dir = 'c:/users/ximzz/desktop/face/crop'
height=[]
width=[]
for root, dirs, files in os.walk(search_dir):

    for file in files:

        image_files = 'c:/users/ximzz/desktop/face/crop/'+file
        image = cv2.imread(image_files)
        height.append(image.shape[0])
        width.append(image.shape[1])

In [65]:
print(int(np.mean(height)))
print(int(np.mean(width)))

891
696


In [66]:
for root, dirs, files in os.walk(search_dir):

    for file in files:

        image_files = 'c:/users/ximzz/desktop/face/crop/'+file
        image = cv2.imread(image_files)
        resized = cv2.resize(image,(400,243))
        cv2.imwrite('c:/users/ximzz/desktop/face/crop/resize/'+file, resized)

# load dataset

In [5]:
face_data = []
for root, dirs, files in os.walk('c:/users/ximzz/desktop/face/crop/resize'):
    for file in files:
        print(file)
        image_files = 'c:/users/ximzz/desktop/face/crop/resize/'+file
        image = cv2.imread(image_files)
        face_data.append(image)

0162671.jpg
0164780.jpg
0281626.jpg
0282585.jpg
0625584.jpg
1.jpg
10.jpg
11.jpg
12.jpg
1273975.jpg
12741023.jpg
1380735.jpg
1381683.jpg
1452458.jpg
1453511.jpg
16071107.jpg
1672581.jpg
1674610.jpg
2.jpg
3.jpg
4.jpg
5.jpg
6.jpg
7.jpg
8.jpg
9.jpg
9872822.jpg


In [6]:
face_array = np.array(face_data)

In [7]:
face_array.shape

(27, 983, 768, 3)

In [15]:
y = np.zeros((27,2),dtype=np.int32)
y[[5,6,7,8,18,19,20,21,22,23,24,25],[0]] =1


In [19]:
y[[0,1,2,3,4,9,10,11,12,13,14,15,16,17,26],[1]]=1
y

array([[0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [0, 1]])

In [21]:
learning_rate = 0.001
training_epoch= 15
batch_size =100
keep_prob = tf.placeholder(tf.float32)
training= True

X= tf.placeholder(tf.float32, shape=[None, 983,768,3 ])
Y= tf.placeholder(tf.float32, shape=[None, 2])


conv1 = tf.layers.conv2d(inputs = X, filters = 32, kernel_size=[3,3], padding='SAME',
        activation = tf.nn.relu)
pool1 = tf.layers.max_pooling2d(inputs = conv1, pool_size=[2,2],
                        padding = 'SAME', strides=2)
dropout1 = tf.layers.dropout(inputs=pool1, rate=0.7,
                training=training)

# after shape=(492,384)

conv2 = tf.layers.conv2d(inputs =dropout1 , filters = 64, kernel_size=[3,3], padding='SAME',
                                    activation = tf.nn.relu)
pool2 = tf.layers.max_pooling2d(inputs = conv2, pool_size=[2,2],
                                       padding = 'SAME', strides=2)
dropout2 = tf.layers.dropout(inputs=pool2, rate=0.7,
                                        training=training) 
# after shape =(246, 192)

conv3 = tf.layers.conv2d(inputs = dropout2, filters = 128, kernel_size=[3,3], padding='SAME',
                                    activation = tf.nn.relu) 
pool3 = tf.layers.max_pooling2d(inputs = conv3, pool_size=[2,2],
                                       padding = 'SAME', strides=2)
dropout3 = tf.layers.dropout(inputs=pool3, rate=0.7,
                                        training=training) #트루면 먹이고 false면 안맥임

#after shape =(123,96)

flat = tf.reshape(dropout3, [-1,123*96*128])
dense4 = tf.layers.dense(inputs = flat, units = 625, activation=tf.nn.relu)
dropout4 = tf.layers.dropout(inputs=dense4,rate=0.5,
                                        training=training)
            
logits = tf.layers.dense(inputs = dropout4, units=2)

cost= tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels=Y))
optimizer= tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

correct_prediction = tf.equal(tf.argmax(logits,1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        
def predict(x_test, training=False):
            return sess.run(logits, feed_dict={X:x_test,
                                                     training:training})
    
    
def get_accuracy(x_test, y_test, training=False):
        return self.sess.run(self.accuracy, feed_dict={X:x_test, Y:y_test, 
                                                       training:training})
    
def train(x_data, y_data, training=True):
        return sess.run([cost, optimizer], feed_dict={X:x_data,
                                                            Y:y_data,
                                                            training:training})

In [22]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range(2000):

    c,_ = sess.run([cost, optimizer], feed_dict={X: face_array, Y:y, keep_prob:0.5})
    print("Epoch :", epoch , "cost : ",c)

Epoch : 0 cost :  231.243
Epoch : 1 cost :  507998.0
Epoch : 2 cost :  246265.0
Epoch : 3 cost :  63255.8
Epoch : 4 cost :  2470.19
Epoch : 5 cost :  21949.2
Epoch : 6 cost :  20994.6
Epoch : 7 cost :  12366.3
Epoch : 8 cost :  6598.73
Epoch : 9 cost :  2942.59
Epoch : 10 cost :  954.159
Epoch : 11 cost :  305.107
Epoch : 12 cost :  264.809
Epoch : 13 cost :  169.739
Epoch : 14 cost :  302.527
Epoch : 15 cost :  315.558
Epoch : 16 cost :  332.309
Epoch : 17 cost :  326.422
Epoch : 18 cost :  309.841
Epoch : 19 cost :  221.541
Epoch : 20 cost :  318.341
Epoch : 21 cost :  194.295
Epoch : 22 cost :  238.955
Epoch : 23 cost :  99.6144
Epoch : 24 cost :  114.575
Epoch : 25 cost :  143.641
Epoch : 26 cost :  87.8698
Epoch : 27 cost :  86.7536
Epoch : 28 cost :  87.3168
Epoch : 29 cost :  53.712
Epoch : 30 cost :  89.9391
Epoch : 31 cost :  48.8513
Epoch : 32 cost :  45.3831
Epoch : 33 cost :  33.263
Epoch : 34 cost :  41.0722
Epoch : 35 cost :  28.7823
Epoch : 36 cost :  38.4579
Epoch : 37 

Epoch : 295 cost :  1.35223
Epoch : 296 cost :  0.366131
Epoch : 297 cost :  1.30861
Epoch : 298 cost :  0.651452
Epoch : 299 cost :  1.25504
Epoch : 300 cost :  3.05843
Epoch : 301 cost :  0.000715272
Epoch : 302 cost :  1.99392
Epoch : 303 cost :  0.307354
Epoch : 304 cost :  3.61898
Epoch : 305 cost :  4.15415
Epoch : 306 cost :  0.689469
Epoch : 307 cost :  0.892004
Epoch : 308 cost :  3.38359
Epoch : 309 cost :  1.3122
Epoch : 310 cost :  2.52095
Epoch : 311 cost :  0.0
Epoch : 312 cost :  1.38013
Epoch : 313 cost :  1.07407
Epoch : 314 cost :  3.23158
Epoch : 315 cost :  1.28884
Epoch : 316 cost :  0.888448
Epoch : 317 cost :  1.18038
Epoch : 318 cost :  0.620015
Epoch : 319 cost :  0.68237
Epoch : 320 cost :  2.55462
Epoch : 321 cost :  2.07714
Epoch : 322 cost :  2.44045
Epoch : 323 cost :  1.01616
Epoch : 324 cost :  0.655609
Epoch : 325 cost :  0.140767
Epoch : 326 cost :  0.826644
Epoch : 327 cost :  0.605758
Epoch : 328 cost :  1.26745
Epoch : 329 cost :  0.742134
Epoch : 3

KeyboardInterrupt: 